# TD-TME SPARQL (utilisateurs Windows)

## Préparation

In [1]:
%endpoint http://localhost:3030/ds
%lang en
%log debug

Endpoint set to: http://localhost:3030/ds
Label preferred languages: ['en']
Logging set to DEBUG

## Exercice 1

### Insertions des données

In [2]:
base <http://example.org/> 
prefix rel: <http://www.perceive.net/schemas/relationship/> 

DELETE DATA
{ GRAPH <http://mlbda/ex1> {}
 }

Success 
 
Update succeeded

In [5]:
base <http://example.org/> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix foaf: <http://xmlns.com/foaf/0.1/> 
prefix rel: <http://www.perceive.net/schemas/relationship/> 

insert data 
{ graph <http://mlbda/ex1> {
<#green-goblin>
    rel:enemyOf <#spiderman> ;
    a foaf:Person ;    # in the context of the Marvel universe
    foaf:name "Green Goblin" .
<#spiderman>
    rel:enemyOf <#green-goblin> ;
    a foaf:Person ;
    foaf:name "Spiderman", "Человек-паук"@ru .
    }
 }

Success 
 
Update succeeded

In [6]:
select (count(distinct ?s) as ?nb_sujets) (count(distinct ?p) as ?nb_props) (count(distinct ?o) as ?nb_objets)
from <http://mlbda/ex1>
where {?s ?p ?o}


nb_sujets,nb_props,nb_objets
2,3,6


## Exercice 2

In [11]:
base <http://example.org/> 

DELETE DATA
{ GRAPH <http://mlbda/ex2> {}
 }

Success 
 
Update succeeded

In [12]:
prefix : <http://example.org/> 

insert data 
{ graph <http://mlbda/ex2> {
:P1    :name        "john" .
:P2    :name        "rick" .
:P3    :name       "lars" .
:P4    :name        "liz" .
:P1    :phone        "11-345-89" .
:P2    :email        "rick@org" .
:P3    :webpage    "www.lars.com" .
:P4    :email        "liz@aut.gov" .
:P4    :phone        "09-134-21" .
:P4    :webpage    "www.liz.pers" 
    }
}

Success 
 
Update succeeded

In [13]:
select (count(distinct ?s) as ?nb_sujets) (count(distinct ?p) as ?nb_props) (count(distinct ?o) as ?nb_objets)
from <http://mlbda/ex2>
where {?s ?p ?o}

nb_sujets,nb_props,nb_objets
4,4,10


### Evaluation des motifs

In [79]:
prefix : <http://example.org/>
select *
from <http://mlbda/ex2>
where {?s ?p ?o}

s,p,o
http://example.org/P2,http://example.org/email,rick@org
http://example.org/P2,http://example.org/name,rick
http://example.org/P1,http://example.org/name,john
http://example.org/P1,http://example.org/phone,11-345-89
http://example.org/P3,http://example.org/name,lars
http://example.org/P3,http://example.org/webpage,www.lars.com
http://example.org/P4,http://example.org/email,liz@aut.gov
http://example.org/P4,http://example.org/name,liz
http://example.org/P4,http://example.org/phone,09-134-21
http://example.org/P4,http://example.org/webpage,www.liz.pers


#### 1-a- les personnes ayant un email et une page web

In [63]:
prefix : <http://example.org/>
select *
from <http://mlbda/ex2>
where {?p :email ?e . ?p :webpage ?w}


p,e,w
http://example.org/P4,liz@aut.gov,www.liz.pers


#### 1-b- les personnes ayant soit un email soit une page web soit les deux

In [64]:
prefix : <http://example.org/>
select *
from <http://mlbda/ex2>
where {{?p :email ?e} UNION {?p :webpage ?w}}

p,e,w
http://example.org/P2,rick@org,
http://example.org/P4,liz@aut.gov,
http://example.org/P3,,www.lars.com
http://example.org/P4,,www.liz.pers


#### 1-c-  les personnes ayant nécessairement un email et éventuellement une page web qu’on retourne

In [65]:
prefix : <http://example.org/>
select *
from <http://mlbda/ex2>
where {{?p :email ?e} OPTIONAL {?p :webpage ?w}}

p,e,w
http://example.org/P2,rick@org,
http://example.org/P4,liz@aut.gov,www.liz.pers


#### 2-les personnes ayant nécessairement un nom éventuellement un email ou une page web


In [66]:
prefix : <http://example.org/>
select *
from <http://mlbda/ex2>
where {?p :name ?n OPTIONAL {?p :email ?e} OPTIONAL {?p :webpage ?w}}

p,n,e,w
http://example.org/P2,rick,rick@org,
http://example.org/P1,john,,
http://example.org/P3,lars,,www.lars.com
http://example.org/P4,liz,liz@aut.gov,www.liz.pers


#### 3- les personnes ayant un nom avec leur email s’ils en ont un et si c’est le cas retourner leur page web s’ils en ont une

In [68]:
prefix : <http://example.org/>
select *
from <http://mlbda/ex2>
where {?p :name ?n OPTIONAL { ?p :email ?e OPTIONAL {?p :webpage ?w} }}

p,n,e,w
http://example.org/P2,rick,rick@org,
http://example.org/P1,john,,
http://example.org/P3,lars,,
http://example.org/P4,liz,liz@aut.gov,www.liz.pers


#### 4- les personnes ayant un nom et soit un email soit un telephone soit les deux

In [73]:
prefix : <http://example.org/>
select *
from <http://mlbda/ex2>
where {?p :name ?n . {{?p :email ?e} UNION{?p :phone ?l}}}

p,n,e,l
http://example.org/P2,rick,rick@org,
http://example.org/P1,john,,11-345-89
http://example.org/P4,liz,liz@aut.gov,
http://example.org/P4,liz,,09-134-21


#### 5-a- les personnes ayant une pageweb mais pas de téléphone sans minus

In [74]:
prefix : <http://example.org/>
select *
from <http://mlbda/ex2>
where {?p :webpage ?w OPTIONAL {?p :phone ?l} FILTER ( ! bound(?l))}

p,w,l
http://example.org/P3,www.lars.com,


#### 5-b- les personnes ayant une pageweb mais pas de téléphone avec minus

In [75]:
prefix : <http://example.org/>
select *
from <http://mlbda/ex2>
where{?p :webpage ?w minus {?p :phone ?l}}


p,w
http://example.org/P3,www.lars.com


## Exercice 3

In [ ]:
Pour cet exercice, nous allons interroger le graphe http://mlbda/ex3

In [19]:
delete data {graph <http://mlbda/ex3> {}}

Success 
 
Update succeeded

In [14]:
base <http://example.org/> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
prefix foaf: <http://xmlns.com/foaf/0.1/> 
prefix rel: <http://www.perceive.net/schemas/relationship/> 
prefix : <http://example.org/> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
prefix univ: <http://www.faculty.ac> 

insert data 
{graph <http://mlbda/ex3>{
#rdf types
:john       a               :Person .
:liz        a               :Person .
:robert     a               :Person .
:dan        a               :Person .
:J.Horrow   a               :Professor .
:larry      a               :Artist .
:cmu        a               :university .
:mit        a               :institute .
:mi         a               :state .
:nyc        a               :City .
:pittsburgh a               :City .
:liz        :friend         :dan .
:liz        :friend         :larry .
:larry      :friend            :dan .
:dan        :friend         :john .

#family and friendships
:john       :hasMother      :suzan .
:john       :hasFather      :richard .
:richard    :hasBrother     :luke .
:john       :hasBrother     :richard .
:larry      :hasFather      :john .
:liz        :hasBrother     :robert .
:liz        :hasFather      :luke .
:liz        :hasMother      :monica .
:larry      :friend         :liz .
:dan        rel:ChildOf     univ:J.Horrow .
:Jim        rel:ChildOf     univ:L.Yang .
:J.Horrow   :studiedAt      :ucla .

#POI
:robert     :livesIn        :nyc .
:liz        :livesIn        :westLafayette .

#degree and university
:luke       :hasDegree      "Eng" .
:luke       :studiedAt      :cmu .
:liz        :studiedAt      :cmu .
:richard    :studiedAt      :mit .
:suzan      :studiedAt      :ucsd .
:monica     :supervisedBy   univ:L.Yang .
:monica     :supervisedBy   univ:J.Horrow .
:Jim        :studiedAt      :ucsb .
:monica     :studiedAt      :cmu .
:john       :studiedAt      :ucsd .
:larry      :studiedAt      :ucsd . 

#misc descriptions

:ucsb       :locatedAt      :santaBarbara .
:cmu        :locatedAt      :pittsburgh .

}}

Success 
 
Update succeeded

In [15]:
select (count(distinct ?s) as ?nb_sujets) (count(distinct ?p) as ?nb_props) (count(distinct ?o) as ?nb_objets)
from <http://mlbda/ex3>
where {?s ?p ?o}

nb_sujets,nb_props,nb_objets
17,11,28


### 1-a Extraire l’ensemble des IRI des sujets

In [76]:
select distinct ?s
from <http://mlbda/ex3>
where {?s ?p ?o}

s
http://example.org/nyc
http://example.org/mi
http://example.org/J.Horrow
http://example.org/monica
http://example.org/liz
http://example.org/cmu
http://example.org/larry
http://example.org/ucsb
http://example.org/luke
http://example.org/robert


### 1-b Extraire l’ensemble des sujets avec leur noms locaux seulement


In [77]:
BASE <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rel: <http://www.perceive.net/schemas/relationship/> 
PREFIX : <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX univ: <http://www.faculty.ac>

select distinct ?s
from <http://mlbda/ex3>
where {?s ?p ?o}

s
http://example.org/nyc
http://example.org/mi
http://example.org/J.Horrow
http://example.org/monica
http://example.org/liz
http://example.org/cmu
http://example.org/larry
http://example.org/ucsb
http://example.org/luke
http://example.org/robert


### 2 Même question avec les prédicats.

In [81]:
select distinct ?p
from <http://mlbda/ex3>
where {?s ?p ?o}

p
http://www.w3.org/1999/02/22-rdf-syntax-ns#type
http://example.org/studiedAt
http://example.org/supervisedBy
http://example.org/friend
http://example.org/livesIn
http://example.org/hasFather
http://example.org/hasMother
http://example.org/hasBrother
http://example.org/locatedAt
http://example.org/hasDegree


### 3 Extraire les villes citées dans des triples de cette base.

In [86]:
BASE <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rel: <http://www.perceive.net/schemas/relationship/> 
PREFIX : <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX univ: <http://www.faculty.ac>

select 
from <http://mlbda/ex3>
where {
    
    
}


y
http://example.org/westLafayette
http://example.org/nyc
http://example.org/pittsburgh
http://example.org/santaBarbara


### 4 Extraire les personnes qui ont étudié dans la même université que l’un de leur parents. (Ne pas considérer les triplets avec la prédicat childOf).

In [87]:
BASE <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rel: <http://www.perceive.net/schemas/relationship/> 
PREFIX : <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX univ: <http://www.faculty.ac>

select  
from <http://mlbda/ex3>
where { 
}

x
http://example.org/liz
http://example.org/larry
http://example.org/john


### 5 Extraire les personnes qui ont étudié dans une université où leurs deux parents ont étudié.

In [89]:
BASE <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rel: <http://www.perceive.net/schemas/relationship/> 
PREFIX : <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX univ: <http://www.faculty.ac>

select  
from <http://mlbda/ex3>
where { 
    }

x
http://example.org/liz


### 6-a Extraire les personnes qui ont étudié dans une université différente de celle de leur père et leur mère 

In [93]:
BASE <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rel: <http://www.perceive.net/schemas/relationship/> 
PREFIX : <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX univ: <http://www.faculty.ac>


select  
from <http://mlbda/ex3>
where
{  }

x


In [94]:
BASE <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rel: <http://www.perceive.net/schemas/relationship/> 
PREFIX : <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX univ: <http://www.faculty.ac>


select  
from <http://mlbda/ex3>
where {  }

x
http://example.org/J.Horrow
http://example.org/monica
http://example.org/luke
http://example.org/suzan
http://example.org/Jim
http://example.org/richard


### 7 Extraire les personnes qui ont un étudié dans la même université qu’un frère ou une soeur.

In [95]:
BASE <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rel: <http://www.perceive.net/schemas/relationship/> 
PREFIX : <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX univ: <http://www.faculty.ac>

SELECT  
from <http://mlbda/ex3>
where {  }

x,univ


### (Question subsidiaire) Donnez les noms et les universités des personnes qui ont au moins un frère ou une soeur et qui n'ont pas de frère ou de soeur qui ont étudié à la même université qu'elles.

In [96]:
BASE <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rel: <http://www.perceive.net/schemas/relationship/> 
PREFIX : <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX univ: <http://www.faculty.ac>

SELECT  
from <http://mlbda/ex3>
WHERE
{
}

x,u
http://example.org/liz,http://example.org/cmu
http://example.org/john,http://example.org/ucsd
http://example.org/richard,http://example.org/mit


### 8 Extraire les personnes qui étudient dans une ville différente de celle où ils habitent.

In [97]:
BASE <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rel: <http://www.perceive.net/schemas/relationship/> 
PREFIX : <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX univ: <http://www.faculty.ac>

select  
from <http://mlbda/ex3>
where { }

x
http://example.org/liz


In [98]:
BASE <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rel: <http://www.perceive.net/schemas/relationship/> 
PREFIX : <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX univ: <http://www.faculty.ac>



select  
from <http://mlbda/ex3>
where 
{
 
}

x
http://example.org/liz


### 9 Extraire les personnes qui sont ami(e)s d’un(e) ami(e) de Liz. Remarque : ne pas retourner les ami(e)s direct(e)s de Liz!

In [101]:
BASE <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> 
PREFIX foaf: <http://xmlns.com/foaf/0.1/>
PREFIX rel: <http://www.perceive.net/schemas/relationship/> 
PREFIX : <http://example.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 
PREFIX univ: <http://www.faculty.ac>

SELECT   
from <http://mlbda/ex3>
where {
 
}

y
http://example.org/john


## Exercice 4

In [17]:
base <http://example.org/> 

DELETE DATA
{ GRAPH <http://mlbda/ex4> {}
 }

Success 
 
Update succeeded

In [20]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#> 
prefix : <http://www.abc.org/> 

insert data 
{
    graph <http://mlbda/ex4>
{
    #rdf types
:john       a               f:emp .
:liz        a               f:emp .
:dan        a               f:emp .
:larry      a               f:emp .
:jim        a               f:emp .
:clark      a               f:emp .
:robert     a               f:emp .

#job
:john   f:Job    "architect" .
:liz    f:Job    "doctor" .
:dan    f:Job    "engineer" .
:larry  f:Job    "singer" .

#surnames
:john       foaf:surname    "john" .
:liz        foaf:surname    "liz" .
:dan        foaf:surname    "dan" .
:larry      foaf:surname    "larry" .

#salary
:john   f:Sal   "33"    .
:liz    f:Sal   "42"    .
:robert f:Sal   "48"    .
:dan    f:Sal  "33"    .

#departement
:john   f:Dept    "urbanism" .
:liz    f:Dept    "surgery" .
:dan    f:Dept    "chemistry" .
:larry  f:Dept    "culture" .
:john  f:Dept    "culture" .

:urbanism   f:Location   "NE1" .
:surgery    f:Location   "SW1" .
:chemistry  f:Location   "Westminster" .
:culture    f:Location   "Slough" .

#manager
:john   f:Mgr   :jim   .
:john   f:Mgr   :clark   .
:liz    f:Mgr   :robert   .
}}

Success 
 
Update succeeded

In [22]:
select (count(distinct ?s) as ?nb_sujets) (count(distinct ?p) as ?nb_props) (count(distinct ?o) as ?nb_objets)
from <http://mlbda/ex4>
where {?s ?p ?o}

nb_sujets,nb_props,nb_objets
11,7,23


In [102]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>

select *
from <http://mlbda/ex4>
where {?s ?p ?o}

s,p,o
http://www.abc.org/dan,http://www.cems.uwe.ac.uk/empdept/concept/Job,engineer
http://www.abc.org/dan,http://www.cems.uwe.ac.uk/empdept/concept/Sal,33
http://www.abc.org/dan,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.cems.uwe.ac.uk/empdept/concept/emp
http://www.abc.org/dan,http://xmlns.com/foaf/0.1/surname,dan
http://www.abc.org/dan,http://www.cems.uwe.ac.uk/empdept/concept/Dept,chemistry
http://www.abc.org/john,http://www.cems.uwe.ac.uk/empdept/concept/Job,architect
http://www.abc.org/john,http://www.cems.uwe.ac.uk/empdept/concept/Sal,33
http://www.abc.org/john,http://www.w3.org/1999/02/22-rdf-syntax-ns#type,http://www.cems.uwe.ac.uk/empdept/concept/emp
http://www.abc.org/john,http://www.cems.uwe.ac.uk/empdept/concept/Mgr,http://www.abc.org/jim
http://www.abc.org/john,http://www.cems.uwe.ac.uk/empdept/concept/Mgr,http://www.abc.org/clark


### 1. Noms des employés avec leur job suivant l’ordre alphabétique de leur nom

In [103]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>

select   
from <http://mlbda/ex4>
where { 
}


name
dan
john
larry
liz


### 2. Les 3 premiers employés suivant l’ordre alphabétique

In [105]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>

select   
from <http://mlbda/ex4>
where {  }
 

ename
dan
john
larry


### 3. Les 3 premiers employés, ayant un surname, les mieux payés

In [106]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>


select  
from <http://mlbda/ex4>
where {  }
 

ename,sal
liz,42
dan,33
john,33


### 4. Les employés qui ne sont ni médecin ni chanteur

In [107]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>


select   
from <http://mlbda/ex4>
where {
 
}


emp,job
http://www.abc.org/dan,engineer
http://www.abc.org/john,architect


### 5. Les employés dont le nom commence par ‘l’ et termine par ‘ry’.

In [110]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>


select   
from <http://mlbda/ex4>
where {
 
}


emp,ename
http://www.abc.org/larry,larry


### 6. a. Le plus grand salaire  sans utiliser ORDER BY et LIMIT 

In [111]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>



select  
from <http://mlbda/ex4>
where {  
}

maxemp,maxsal
http://www.abc.org/robert,48


### 6. b Le plus grand salaire en utilisant ORDER BY et LIMIT

In [112]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>


SELECT  
from <http://mlbda/ex4>
WHERE { 
}
 

maxemp,maxsal
http://www.abc.org/robert,48


### 7. Les couples d'employés qui gagnent le même salaire (chaque couple d'employés doit apparaître une seule fois).

In [113]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>

select   
from <http://mlbda/ex4>
where { 
}

emp1,sal,emp2
http://www.abc.org/dan,33,http://www.abc.org/john


### 8. Les couples d'employés tels que la différence entre leurs salaires est supérieure à 5. 
Indice : utiliser la function xs:integer( ?var) afin de convertir un literal en un entirer. 
Pour cela, ne pas oublier d’inclure la ligne suivante dans votre fichier requêtes


In [114]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>

select  
from <http://mlbda/ex4>
where {
}

emp1,sal1,emp2,sal2
http://www.abc.org/liz,42,http://www.abc.org/dan,33
http://www.abc.org/liz,42,http://www.abc.org/john,33
http://www.abc.org/robert,48,http://www.abc.org/dan,33
http://www.abc.org/robert,48,http://www.abc.org/john,33
http://www.abc.org/robert,48,http://www.abc.org/liz,42


### 9.Le nombre de départements

In [115]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>

select  
from <http://mlbda/ex4>
where {  
}

count
4


### 10. Le nombre d’employés par département

In [116]:
prefix foaf: <http://xmlns.com/foaf/0.1/>
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#>
prefix f: <http://www.cems.uwe.ac.uk/empdept/concept/>
prefix xs: <http://www.w3.org/2001/XMLSchema#>


select  
from <http://mlbda/ex4>
where {  
} 

dept,count
urbanism,1
surgery,1
culture,2
chemistry,1
